# Treinamento e validação de Modelos

In [1]:
import os
import sagemaker
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix, hstack
from python_scripts.save_load import load_df_from_bucket, save_df_to_s3_bucket, save_to_s3_bucket_as_libsvm, BUCKET_MODEL
from python_scripts.modelling import create_train_validation_test_sets, setup_model, make_prediction    

### Processamento adicional
Para utilizar os dados obtidos para treinamento de um modelo, algumas etapas adicionais de processamento serão executadas:
- Apenas as colunas de lemmas, tamanho médio de sentença e tamanho médio de palavra serão utilizadas no modelo
- Separação de dados de treino, validação e teste
- Transformação TF-IDF será aplicada à coluna de lemmas
- Colunas de tamanhos de sentenças e palavras serão padronizadas (ie RobustScaler para evitar problemas com outliers)
- Salvar arquivos referentes aos dados de treino, validação e teste após este processamento

In [2]:
full_df = load_df_from_bucket('dados_processados.csv', tipo='processado')
full_df.head()

,fake,text,words,lemmas,avg_sent_len,avg_word_len,words_str,lemmas_str
0,0,A divisão do STF ao meio entre partidários e ...,"['divisão', 'STF', 'meio', 'partidários', 'ind...","['divisão', 'STF', 'meio', 'partidário', 'inde...",10.747664,6.690641,divisão STF meio partidários independentes fic...,divisão STF meio partidário independente ficar...
1,1,"General manda recado para STF: ""Abaixaram as c...","['General', 'manda', 'recado', 'STF', 'Abaixar...","['general', 'mandar', 'recado', 'STF', 'abaixa...",11.000000,6.461584,General manda recado STF Abaixaram calças Cong...,general mandar recado STF abaixar calça congre...
2,1,O Nordeste acordou! Lula e o PT são enxotados:...,"['Nordeste', 'acordou', 'Lula', 'PT', 'enxotad...","['nordeste', 'acordar', 'Lula', 'PT', 'enxotar...",7.333333,6.565873,Nordeste acordou Lula PT enxotados Chega bande...,nordeste acordar Lula PT enxotar chegar bandei...
3,0,"Dois relatórios da Polícia Federal, com análi...","['Dois', 'relatórios', 'Polícia', 'Federal', '...","['dois', 'relatório', 'Polícia', 'Federal', 'a...",17.000000,7.237319,Dois relatórios Polícia Federal análises mater...,dois relatório Polícia Federal análise materia...
4,1,Coreia do Norte declara status de QUASE-GUERRA...,"['Coreia', 'Norte', 'declara', 'status', 'QUAS...","['Coreia', 'Norte', 'declarar', 'status', 'QUA...",11.666667,6.513799,Coreia Norte declara status QUASE-GUERRA mobil...,Coreia Norte declarar status QUASE-GUERRA mobi...


In [3]:
model_df = full_df[['fake', 'lemmas_str', 'avg_sent_len', 'avg_word_len']]
model_df.head()

,fake,lemmas_str,avg_sent_len,avg_word_len
0,0,divisão STF meio partidário independente ficar...,10.747664,6.690641
1,1,general mandar recado STF abaixar calça congre...,11.000000,6.461584
2,1,nordeste acordar Lula PT enxotar chegar bandei...,7.333333,6.565873
3,0,dois relatório Polícia Federal análise materia...,17.000000,7.237319
4,1,Coreia Norte declarar status QUASE-GUERRA mobi...,11.666667,6.513799


## Modelo 1: Baseado em tamanho de sentenças e palavras

O primeiro modelo treinado é um modelo mais simples. Aqui, apenas as colunas referentes ao tamanho médio de sentença e de palavra de cada artigo são consideradas, sem se levar em conta  o conteúdo da notícia propriamente dita.

### Separar dados de treino e teste

In [4]:
train_1, test_1, validate_1 = create_train_validation_test_sets(model_df.drop('lemmas_str', axis=1), 
                                                                stratify_col='fake',
                                                                test_size=0.2, random_state=42)

In [5]:
def create_x_y_1(base_df, target_col='fake'):
    return base_df.drop(target_col, axis=1), base_df[target_col]

x_train_1, y_train_1 = create_x_y_1(train_1)
x_validate_1, y_validate_1 = create_x_y_1(validate_1)
x_test_1, y_test_1 = create_x_y_1(test_1)

### Upload de dados para o S3

In [6]:
file_name_tuples = [(x_train_1, y_train_1, 'train'), 
                   (x_test_1, y_test_1, 'test'), 
                   (x_validate_1, y_validate_1, 'validate')]

for x, y, prefix in file_name_tuples:
    save_to_s3_bucket_as_libsvm(x, y, prefix=prefix, filename='model_1.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazon

### Treinar modelo

In [7]:
xgb_model, data_channels = setup_model(base_image='xgboost', model_name='model_1', instance_count=2, 
                                       instance_type='ml.m4.xlarge')
xgb_model.fit(inputs=data_channels)

print('ready for hosting!')

2022-11-11 23:30:14 Starting - Starting the training job...ProfilerReport-1668209414: InProgress
...
2022-11-11 23:31:08 Starting - Preparing the instances for training...............
2022-11-11 23:33:46 Downloading - Downloading input data...
2022-11-11 23:34:16 Training - Training image download completed. Training in progress....[2022-11-11 23:34:31.893 ip-10-0-68-94.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-11:23:34:31:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-11:23:34:31:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2022-11-11:23:34:31:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-11-11:23:34:31:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-11:23:34:31:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-11:23:34:31:INFO] files path: /opt/ml/input/data/train
[2022-11-11:23:34:31:INF

In [8]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                 serializer=sagemaker.serializers.LibSVMSerializer(),
                                 instance_type='ml.m4.xlarge')

--------!

### Métricas no conjunto de testes

In [10]:
y_pred_1 = make_prediction(xgb_predictor, model_name='model_1')

In [11]:
print(classification_report(y_test_1, y_pred_1))

              precision    recall  f1-score   support

           0       0.68      0.69      0.68       360
           1       0.68      0.67      0.68       360

    accuracy                           0.68       720
   macro avg       0.68      0.68      0.68       720
weighted avg       0.68      0.68      0.68       720



Modelo razoável dado sua simplicidade (ignora completamente o contexto)

### Salvar resultados de predição

In [12]:
df_pred_1 = pd.DataFrame({'pred_1': y_pred_1})
df_pred_1.to_csv(f's3://{BUCKET_MODEL}/test/pred_1.csv', encoding='utf-8', index=False)

### Encerrar modelo
Após pegar métricas e dados de interesse, encerrar modelo para evitar cobranças.

In [13]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True) 

## Modelo 2: TF-IDF

In [16]:
train_2, test_2, validate_2 = create_train_validation_test_sets(model_df.drop(['avg_word_len', 
                                                                               'avg_sent_len'], axis=1), 
                                                                stratify_col='fake',
                                                                test_size=0.2, random_state=42)

### Processamento adicional

Um vetorizador TFIDF é utilizado para converter os dados textuais em colunas do DataFrame.

In [62]:
# Criar vetorizador TFIDF e ajustar aos dados de treinamento
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
tfidf.fit(train_2['lemmas_str'])

TfidfVectorizer(lowercase=False, ngram_range=(1, 2))

In [20]:
def create_x_y_2(base_df, tfidf, target_col='fake', lemma_col = 'lemmas_str'):
    tfidf_res = tfidf.transform(base_df[lemma_col])
    return tfidf_res, base_df[target_col]

x_train_2, y_train_2 = create_x_y_2(train_2, tfidf)
x_validate_2, y_validate_2 = create_x_y_2(validate_2, tfidf)
x_test_2, y_test_2 = create_x_y_2(test_2, tfidf)

### Upload de dados para o S3

In [21]:
file_name_tuples = [(x_train_2, y_train_2, 'train'), 
                   (x_test_2, y_test_2, 'test'), 
                   (x_validate_2, y_validate_2, 'validate')]

for x, y, prefix in file_name_tuples:
    save_to_s3_bucket_as_libsvm(x, y, prefix=prefix, filename='model_2.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazon

### Treinar modelo

In [22]:
xgb_model_2, data_channels_2 = setup_model(base_image='xgboost', model_name='model_2', instance_count=4, 
                                           instance_type='ml.m4.xlarge')
xgb_model_2.fit(inputs=data_channels_2)

print('ready for hosting!')

2022-11-12 00:05:33 Starting - Starting the training job...ProfilerReport-1668211533: InProgress
..................
2022-11-12 00:08:47 Starting - Preparing the instances for training.....................
2022-11-12 00:12:34 Downloading - Downloading input data.....[2022-11-12 00:13:17.428 ip-10-2-89-124.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-12:00:13:17:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-12:00:13:17:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2022-11-12:00:13:17:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-11-12:00:13:17:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-12:00:13:17:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-12:00:13:17:INFO] files path: /opt/ml/input/data/train
[2022-11-12:00:13:17:INFO] files path: /opt/ml/input/data/validation
[2022-11-12:00:13:17:I

In [23]:
xgb_predictor_2 = xgb_model_2.deploy(initial_instance_count=1,
                                     serializer=sagemaker.serializers.LibSVMSerializer(),
                                     instance_type='ml.m4.xlarge')

-------!

### Métricas do modelo

In [ ]:
# Muito grande! Ver algo de batch ou como customizar tamanho máximo de payload. 
# Por ora, fazer tosquice pra contonar e obter resultados preliminares
# y_pred_2 = make_prediction(xgb_predictor_2, model_name='model_2')

In [25]:
half = test_2.shape[0]//2
x_test_2_pt1, y_test_2_pt1 = create_x_y_2(test_2.iloc[:half], tfidf)
x_test_2_pt2, y_test_2_pt2 = create_x_y_2(test_2.iloc[half:], tfidf)

In [30]:
save_to_s3_bucket_as_libsvm(x_test_2_pt1, y_test_2_pt1, prefix='test', 
                            filename='model_2_pt1.libsvm', tipo='modelo')
save_to_s3_bucket_as_libsvm(x_test_2_pt2, y_test_2_pt2, prefix='test', 
                            filename='model_2_pt2.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [34]:
import s3fs
from sklearn.datasets import load_svmlight_file

fs = s3fs.S3FileSystem()
s3_path = f's3://projetointerdisciplinartreinoteste/test/model_2_pt1.libsvm'

with fs.open(s3_path) as libsvm_file:
    y_temp_1 = xgb_predictor_2.predict(libsvm_file)


s3_path = f's3://projetointerdisciplinartreinoteste/test/model_2_pt2.libsvm'

with fs.open(s3_path) as libsvm_file:
    y_temp_2 = xgb_predictor_2.predict(libsvm_file)


In [40]:
y_pred_list_1 = [1 if float(x) >= 0.5 else 0 for x in y_temp_1.decode('utf-8').split('\n') if x != '']
y_pred_list_2 = [1 if float(x) >= 0.5 else 0 for x in y_temp_2.decode('utf-8').split('\n') if x != '']

In [41]:
y_pred_2 = y_pred_list_1+y_pred_list_2

In [43]:
print(classification_report(y_test_2, y_pred_2))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       360
           1       0.94      0.98      0.96       360

    accuracy                           0.96       720
   macro avg       0.96      0.96      0.96       720
weighted avg       0.96      0.96      0.96       720



## TODO arrumar comentários
Resultados excelentes, diria até suspeitos!

## Salvar resultados da predição

In [50]:
df_pred_2 = pd.DataFrame({'pred_2': y_pred_2})
df_pred_2.to_csv(f's3://{BUCKET_MODEL}/test/pred_2.csv', encoding='utf-8', index=False)

## Teste adicional - Segundo dataset
Um segundo dataset de notícias também foi encontrado: https://github.com/Gabriel-Lino-Garcia/FakeRecogna
Para validar os excelentes resultados obtido no teste com o dataset original, o modelo também foi testado neste dataset.

Os dados deste dataset já vem no formato de lemma e sem stopwords, de forma que o vetorizador TFIDF pode ser aplicado diretamente a ele.

In [76]:
dset_link = 'https://github.com/Gabriel-Lino-Garcia/FakeRecogna/raw/master/dataset/FakeRecogna.xlsx'
alternative_df = pd.read_excel(dset_link, engine='openpyxl')  # engine para xlsx
alternative_df.head()

,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,Boato – Ocorreu um apagão no Vaticano. O papa ...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,\nEdgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,NaN,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,27/03/2020 18h25,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,NaN,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,07/08/2020 13h42,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,NaN,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,27/04/2020 14h53,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,NaN,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


#### Nulos
Verificar presença de linha com campo 'Noticia' nulo e remover se for o caso

In [77]:
alternative_df.isna().sum()

Titulo         31
Subtitulo    6323
Noticia         1
Categoria       1
Data          352
Autor          17
URL             1
Classe          1
dtype: int64

In [78]:
alternative_df[alternative_df['Noticia'].isna()]

,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
7337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
alternative_df.dropna(subset=['Noticia'], inplace=True)

#### Preparação do segundo dataset

Colunas de interesse são apenas 'notícia' (equivalente a 'lemma_str') e classe (equivalente a 'fake'). Note que, neste dataset, Classe 0.0 indica notícia Falsa e classe 1.0 indica notícia real! Portanto, é necessário adaptar o DataFrame.

In [81]:
alternative_df['lemmas_str'] = alternative_df['Noticia']
alternative_df['fake'] = (~alternative_df['Classe'].astype(bool)).astype(int) # inverter 1 e 0

# Dropar colunas agora redundantes
alternative_df = alternative_df.drop(['Noticia', 'Classe'], axis=1)

Amostrar 400 linhas para teste

In [83]:
n = 400
sample_df = alternative_df.sample(n=n, random_state=42)

#### Predição

Aplicar TF-IDF para vetorizar, salvar dados no S3 e efetuar predição

In [85]:
x_test_alt, y_test_alt = create_x_y_2(sample_df, tfidf)
save_to_s3_bucket_as_libsvm(x_test_alt, y_test_alt, 
                            prefix='test', filename='model_2_alt.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [86]:
y_pred_alt = make_prediction(xgb_predictor_2, model_name='model_2_alt')

#### Métricas

In [87]:
print(classification_report(y_test_alt, y_pred_alt))

              precision    recall  f1-score   support

           0       1.00      0.01      0.02       205
           1       0.49      1.00      0.66       195

    accuracy                           0.49       400
   macro avg       0.74      0.50      0.34       400
weighted avg       0.75      0.49      0.33       400



#### Salvar predição

In [88]:
df_pred_alt = pd.DataFrame({'pred_alt': y_pred_alt})
df_pred_alt.to_csv(f's3://{BUCKET_MODEL}/test/pred_alt.csv', encoding='utf-8', index=False)

### Encerrar modelo
Após pegar métricas e dados de interesse, encerrar modelo para evitar cobranças.

In [89]:
xgb_predictor_2.delete_endpoint(delete_endpoint_config=True) 